In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum


class Production_data:
    def __init__(self, number_of_periods, number_of_items, demand_forecast, production_cost, holding_cost, setup_cost, item_requirements , capacity):
        self.T=number_of_periods
        self.items=number_of_items
        self.demand_forecast=np.array(demand_forecast)
        self.production_cost=np.array(production_cost)
        self.holding_cost=np.array(holding_cost)
        self.setup_cost=np.array(setup_cost)
        self.item_requirements=np.array(item_requirements)
        self.capacity=np.array(capacity)

file_name = "CLSP+ST-instances Data-R.xlsx"
#file_name = "prova2.xlsx"

xls = pd.ExcelFile(file_name)  # Read the whole file

tables_keywords = ["Demand Forecast:", "Production Cost", "Holding Cost", "Setup Cost", "UnitsOfCapacity", "Capacity"]


In [4]:

def read_data(xls, sheet_name):
    tables_dict = {}
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Define which column to check for each keyword
    columns_to_check = {
        "Demand Forecast:": 0,
        "Production Cost": 0,
        "Holding Cost": 0,
        "Setup Cost": 0,
        "UnitsOfCapacity": 1,  # Check the second column for this keyword
        "Capacity": 0
    }
    
    # Iterate through the keywords to find each table
    for keyword in tables_keywords:
        column_idx = columns_to_check.get(keyword, 0)
        
        # Check in the specified column for the keyword
        match = df[df.iloc[:, column_idx].astype(str).str.contains(keyword, na=False)]
        
        if not match.empty:
            table_start_row = match.index[0] + 1
            
            # Find the end of the current table (next keyword or empty rows)
            end_row = None
            for next_keyword in tables_keywords:
                if next_keyword != keyword:
                    next_column_idx = columns_to_check.get(next_keyword, 0)
                    next_match = df.loc[table_start_row:][df.loc[table_start_row:].iloc[:, next_column_idx].astype(str).str.contains(next_keyword, na=False)]
                    if not next_match.empty:
                        potential_end = next_match.index[0]
                        if end_row is None or potential_end < end_row:
                            end_row = potential_end
            
            # If no next keyword found, look for empty rows
            if end_row is None:
                for i in range(table_start_row, len(df)):
                    # Check if row is empty or contains only NaN values
                    if df.iloc[i].isna().all():
                        end_row = i
                        break
            
            # If still no end found, use the end of the dataframe
            if end_row is None:
                end_row = len(df)
            
            # Extract the table
            table_df = df.iloc[table_start_row:end_row]
            
            # Remove completely empty rows
            table_df = table_df.dropna(how='all')
            
            # Remove completely empty columns
            table_df = table_df.dropna(axis=1, how='all')
            
            # Remove any remaining NaN values by filling with 0
            table_df = table_df.fillna(0)
            
            # Convert to numpy array
            table = table_df.to_numpy()
            tables_dict[keyword] = table
    
    # Create an instance of Production_data class
    production_data = Production_data(
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[1]-1,
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[0]-1,  
        tables_dict.get("Demand Forecast:", np.zeros((1,1))),  
        tables_dict.get("Production Cost", np.zeros((1,1))),  
        tables_dict.get("Holding Cost", np.zeros((1,1))),  
        tables_dict.get("Setup Cost", np.zeros((1,1))),  
        tables_dict.get("UnitsOfCapacity", np.zeros((1,1))),  
        tables_dict.get("Capacity", np.zeros((1,1)))

    )
    
    production_data.capacity = np.vstack([
    np.zeros((1, production_data.capacity.shape[1]), dtype=production_data.capacity.dtype),production_data.capacity])

    production_data.item_requirements = np.vstack([
    np.zeros((1, production_data.item_requirements.shape[1]), dtype=production_data.item_requirements.dtype),production_data.item_requirements
])
    return production_data 

#data=read_data(xls, "Data-20-12 (1)")

#print(data.T)
#print(data.items)

#print(data.demand_forecast)
#print(data.holding_cost)
#print(data.setup_cost)
#print(data.capacity)
#print(data.production_cost)

#print(data.capacity[0, 1])
#print(data.capacity[12, 1])
#print(data.item_requirements)

sheet_list=["Data-20-12 (1)"]#, "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)" , "Data-100-24 (1)", "Data-100-24 (2)",
          #  "Data-200-24" ]

#sheet_list=["Sheet1"]





In [9]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum

def solve_MILP(data, sheet_name, results_df):

    model=gp.Model("MiCLSP-ST")

    T=data.T
    O=data.items

    #DUMMY AT PERIOD 0?

    #decision variables, constraint 5 and 6 added here
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y" ) #produce of not for this item in this period
    s=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t

    model.setObjective(
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
        GRB.MINIMIZE
    )  


    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(s[i, t-1]+x[i, t]-s[i, t]==data.demand_forecast[i, t])

    

    #constraint 3, this definitely works
    for t in range(1, T+1):
        print(data.capacity[t,1])
        model.addConstr( quicksum( x[i, t]*data.item_requirements[i, 1] + data.item_requirements[i,2] * y[i,t] for i in range(1, O+1))<= data.capacity[t, 1])
        
    #constraint 4, this defintely ok
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    #constraint 7, no initial inventory for all items, this definitely works
    for i in range(1, O+1):
        model.addConstr( s[i, 0] == 0, name="no init inv") #lloks like this constrint is ok
        model.addConstr( s[i, T] == 0, name="no final inv")
    
    #solve the model
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        print("\noptimal solution found:")
        print(model.ObjVal)

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Inventory of item 1": s[1, t].X,
                "Produced for item 1": x[1, t].X
            })
        
        # Add all periods at once to the results DataFrame
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)
    
    return results_df
    



    


In [10]:


# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Inventory of item 1", "Produced for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_MILP(data, sheet_name, results_df)

results_df.to_csv("results_MILP.csv", index=False)

C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, s

13430
11390
10200
12920
11900
10370
11390
11730
11050
11900
13430
11220
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 532 rows, 819 columns and 1720 nonzeros
Model fingerprint: 0xc502273b
Variable types: 546 continuous, 273 integer (273 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+04]
Found heuristic solution: objective 101778.20000
Presolve removed 101 rows and 179 columns
Presolve time: 0.01s
Presolved: 431 rows, 640 columns, 1460 nonzeros
Variable types: 420 continuous, 220 integer (220 binary)

Root relaxation: objective 8.197407e+04, 539 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Boun

C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\1025691463.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, sheet_results], ignore_index=True)


### Plant location reformulation

As for the single-item lot-sizing problem, an alternative formulation for the current problem, also 
called plant location reformulation (PLRFIP), is obtained by replacing in the model (MiCLSP-STIP) the 
production variables 𝑥𝑖𝑡 by the variables 𝑤𝑠𝑡 𝑖 given by:

Where 𝑤𝑠𝑡 𝑖 can be interpreted as the portion of demand of item 𝑖 ∈ 𝑃 in period 𝑡 ∈ 𝐻 fulfilled by 
production in period 𝑠 ∈ 𝐻,𝑠 ≤ 𝑡. Write down this reformulation and check the validity of your 
proposed model. 

In [11]:
def solve_PLRF(data, sheet_name, results_df):

    model=gp.Model("PLRF")


    #decisions variables

    T=data.T
    O=data.items

    w=model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name = "w" )#fraction of demand of item i produced in period s to satisfy period t
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y") #produce or not for item i in period i

    model.setObjective(
        #setup cost
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        #production cost, j is actual period and t in future period
        quicksum(quicksum(quicksum(data.production_cost[i, s] * data.demand_forecast[i,t] * w[i, s, t] 
                          for t in range(s, T+1)) # Changed range
                 for s in range(1, T+1))
        for i in range(1, O+1))+
        #holding cost, i per item
        quicksum( quicksum(quicksum
                           (sum(data.holding_cost[ i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j] for j in range(s+1, T+1))
                           for s in range(1, T+1)) for i in range(1, O+1)),
        GRB.MINIMIZE
    )
    
    

    
    # Each demand must be fully satisfied (like each customer must be served) for each item
    for i in range(1, O+1):
        for t in range(1, T+1):
            model.addConstr( quicksum(w[i, s, t] for s in range(1, t+1)) == 1, name= f"demand_{i}{t}")
            
    #cannot produce any fraction of demand in period s is the variable y is 0, for item i
    for i in range(1, O+1):
        for t in range(1, T+1):
            for s in range(1, t+1):
                model.addConstr( w[i, s, t] <= y[i, s] )
        
    #still capacity constraints fossure
    #constraint 3

    for s in range(1, T+1):
        model.addConstr(
            quicksum(
                # Production time: sum over all demands being produced in period s
                sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                # Setup time
                data.item_requirements[i, 2] * y[i, s]
                for i in range(1, O+1)
            ) <= data.capacity[s, 1],
            name=f"capacity_{s}"
        )
        
    
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr( sum( w[i, t, s] for s in range(t, T+1) )-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    
    #solve
    model.optimize()

    if model.status == GRB.OPTIMAL:

        print("\noptimal solution found:")

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Setup Decision": y[1, t].X,
                "Production Quantity": sum(w[2, t, j].X * data.demand_forecast[2, j] for j in range(t, T+1)),
                "Capacity Usage": sum(sum(w[i, t, j].X * data.demand_forecast[i, j] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y[i,t].X  
                                        for j in range(t, T+1)) 
                                    for i in range(1, O+1)) / data.capacity[t, 1],
                "Total Cost": model.objVal
            })
        
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([ results_df, sheet_results ], ignore_index=False)


    return results_df


In [12]:

# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Produced in the period for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_PLRF(data, sheet_name, results_df)

results_df.to_csv("results_PLRF.csv", index=False)

C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_17108\384112948.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, s

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2052 rows, 3822 columns and 8280 nonzeros
Model fingerprint: 0x098c81b5
Variable types: 3549 continuous, 273 integer (273 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 521 rows and 2082 columns
Presolve time: 0.01s
Presolved: 1531 rows, 1740 columns, 5860 nonzeros
Variable types: 1520 continuous, 220 integer (220 binary)
Found heuristic solution: objective 98834.000000

Root relaxation: objective 8.939710e+04, 1792 iterations, 0.04 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Lagrangian relaxation of Constraint (3)

TO FIX, THERE MIGHT BE SOME ISSUES AND I DON'T KNOW, IT SEAMS LIKE THERE IS A OSCILLATION PROBLEM.

In [13]:

def lagrangian_relaxation(xls, sheet_name, results_df, theta=0.10, epsilon=1e-1, lambda_max=1):

    #needed to set a lambda max since withut the we were going to negative values of ioibjective funciton

    model=gp.Model("MiCLSP-ST")

    T=data.T
    O=data.items

    #we start from all lambdas==0 then with subgradient method we move closer to the optimal

    #ok here we need lambda values, we would have T lambda values, and we will use subgradient method
    lambda_vals=np.zeros(T+1)

    max_iterations=100

    best_obj=float("-inf")
    best_solution = None

    for iteration in range(1, max_iterations):
        #decision variables, constraint 5 and 6 added here
        y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y" ) #produce of not for this item in this period
        s=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
        x=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t


        #lower bound ZLR
        model.setObjective(
            quicksum( quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
            quicksum( quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
            quicksum( quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)) +
            quicksum( lambda_vals[t] * ( quicksum( x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i,2] * y[i,t] for i in range(1, O+1)) - data.capacity[t, 1])  for t in range(1, T+1)),
            GRB.MINIMIZE                 #positive
        )  


        #constraint 2
        for t in range(1, T+1):
            for i in range(1, O+1):
                model.addConstr(s[i, t-1]+x[i, t]-s[i, t]==data.demand_forecast[i, t])
            
        #constraint 4, this defintely ok
        for t in range(1, T+1):
            for i in range(1, O+1):
                model.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

        #constraint 7, no initial inventory for all items, this definitely works
        for i in range(1, O+1):
            model.addConstr( s[i, 0] == 0, name="no init inv") #lloks like this constrint is ok
        
        #solve the model
        model.optimize()

        if model.Status == GRB.OPTIMAL:

            print(f"iteration {iteration}: Objective value: {model.ObjVal}")

            #we found a new solution, is it smaller than the actual best solution?
            if model.ObjVal > best_obj:
                best_obj=model.ObjVal
                Best_solution=[ (i, t, x[i, t].X, s[i, t].X, y[i, t].X) for t in range(1, T+1) for i in range(1, O+1)]

            

        
        #now must optimize
        #compute the subgradient, makes sense yes to set them to 0 at each iteration
        g_lambda = np.zeros(T+1)

        for t in range(1, T+1):
            lhs = sum( x[i, t].X * data.item_requirements[i, 1] + data.item_requirements[i,2] * y[i,t].X for i in range(1, O+1))
            g_lambda[t] = lhs - data.capacity[t, 1]
            print(g_lambda[t]) #negative mean the constraint violated

        #step size
        step= theta/iteration

        print("\n")
        
        #now we update lambdas, but we need to keep them positive
        for t in range(1, T+1):
            lambda_vals[t]=max(0, min(lambda_max, lambda_vals[t] + step * g_lambda[t]))
            print(lambda_vals[t])


        #it is the norm of the vector g_lambda
        if np.linalg.norm(g_lambda) < epsilon:
            print("optimal solution found")
            break

    if best_solution:
        results=[]
        for (i, t, x_val, y_val, s_val) in best_solution:
            results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Item": i,
                "Produced": x_val,
                "Inventory": s_val,
                "Setup ": y_val
            })

    sheet_results=pd.DataFrame(results)
    results_df=pd.concat([results_df, sheet_results], ignore_index=True)

    return results_df


            



In [14]:
# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Produced in the period for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = lagrangian_relaxation(data, sheet_name, results_df)

results_df.to_csv("results_LR.csv", index=False)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 819 columns and 1220 nonzeros
Model fingerprint: 0x502b993e
Variable types: 546 continuous, 273 integer (273 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 1e+04]
Found heuristic solution: objective 1.797271e+07
Presolve removed 479 rows and 787 columns
Presolve time: 0.07s
Presolved: 21 rows, 32 columns, 52 nonzeros
Found heuristic solution: objective 7748598.0000
Variable types: 21 continuous, 11 integer (11 binary)

Root relaxation: objective 7.533878e+06, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

C:\Users\ricca\AppData\Local\Temp\ipykernel_22188\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_22188\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_22188\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior

iteration 1: Objective value: 7534824.0
2167928.0
-71771.0
-502711.0
-511324.0
91869.0
9639.0
170316.0
-342418.0
-84887.0
-259671.0
-280536.0
-439694.0


1.0
0.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1000 rows, 1638 columns and 2440 nonzeros
Model fingerprint: 0xd917d580
Variable types: 1092 continuous, 546 integer (546 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 1e+04]

MIP start from previous solve produced solution with objective 8.34794e+06 (0.03s)
Loaded MIP start from previous solve with objective 8.34794e+06

Presolve removed 979 rows and 1606 columns
Presolve time: 0.03s
Presolved: 21 r

UnboundLocalError: cannot access local variable 'results' where it is not associated with a value

## LAGRANGIAN RELAXATIO OF FACILITY REL 

## DANTZIG WOLFE

In [ ]:
#each subproblem is a single item lot sizing problem, the only constriant linking the items is constraint (3)


model=gp.Model("Dantzig-Wolfe")

#decision variables
#number of plans grows dinamically during algorithm
lambda_vars= {(i, p): model.addVar() for i in O for p in current_plans[i]}



## BENDERS DECOMPOSITION

In [ ]:
#yit is the complicating variable, to fix, we fix it so then the problem becomes a simple LP 
#ok is gurobi the one that chooses the values for y
#create dual and if dual feasible we add cut


def Benders_decomposition(data, sheet_name, max_iterations=1):

    #so in the master prob we take a decision jsut for y

    master=gp.Model("Benders")

    T=data.T
    O=data.items

  
    #decision variables, constraint 5 and 6 added here
    y=master.addVars(O+1, T+1, vtype=GRB.BINARY, name="y" ) #produce of not for this item in this period
    theta=master.addVar(vtype=GRB.CONTINUOUS, lb=0, name="theta") 


    #the complete initial problem gets rewritten like this
    #problem P2
    master.setObjective(
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        theta,
        GRB.MINIMIZE
    )  #theta subprob

    #add contraints that involve the variable y!!!!!!!!!!
    #issue: these constraints involve alse s and x, but we cannot take decisons for them here,
    #we need therefore to use surrogate constriants, that are weaker, but subprob will handle them all


    #contraint(3) (surrogate)
    for t in range(1, T+1):
        master.addConstr( quicksum( (data.item_requirements[i,1] + data.item_requirements[i,2]) * y[i,t] for i in range(1, O+1))<= data.capacity[t, 1])
        #so fossure at the beginning we start from y all 0 and that's ok

    best_obj = float('inf')  # Initialize to infinity for minimization
    best_y = None
    
    for iteration in range(max_iterations):
        print(f"Iteration {iteration}:")
        
        # Solve master problem
        master.optimize()

        
        # Get y values
        y_vals = [[0 for t in range(T+1)] for i in range(O+1)]
        for t in range(1, T+1):
            for i in range(1, O+1):
                y_vals[i][t] = y[i, t].X
                if y_vals[i][t]!=0:
                    print(y_vals[i][t])

        
        subprob=create_subprob(y_vals, data)
        subprob.optimize()

        #get the duals
        capacity_duals = {}
        prod_limit_duals = {}
        
        if subprob.status == GRB.OPTIMAL:
            # Get regular dual values for feasible problem
            for t in range(1, T+1):
                constr = subprob.getConstrByName(f"capacity_{t}")
                capacity_duals[t] = constr.Pi
            
            for t in range(1, T+1):
                for i in range(1, O+1):
                    constr = subprob.getConstrByName(f"prod_limit_{i}_{t}")
                    prod_limit_duals[(i,t)] = constr.Pi
                    
        elif subprob.status == GRB.INFEASIBLE:
            # Get Farkas duals for infeasible problem
            try:
                for t in range(1, T+1):
                    constr = subprob.getConstrByName(f"capacity_{t}")
                    capacity_duals[t] = constr.FarkasDual
                
                for t in range(1, T+1):
                    for i in range(1, O+1):
                        constr = subprob.getConstrByName(f"prod_limit_{i}_{t}")
                        prod_limit_duals[(i,t)] = constr.FarkasDual
            except:
                print("Could not get Farkas duals, using IIS instead")
                subprob.computeIIS()
                # Instead of returning, add a feasibility cut based on IIS
                iis_constrs = [c for c in subprob.getConstrs() if c.IISConstr]
                # Add a cut that prevents the same infeasible combination
                master.addConstr(
                    quicksum(y[i,t] for (i,t) in [(i,t) for c in iis_constrs 
                                                 if "prod_limit" in c.ConstrName 
                                                 for i in range(1, O+1) 
                                                 for t in range(1, T+1)]) <= 
                    len([c for c in iis_constrs if "prod_limit" in c.ConstrName]) - 1
                )
                master.optimize()
                continue
        else:
            print(f"Subproblem status: {subprob.status}")
            continue  # Skip this iteration but continue with the next
        
        # Check dual feasibility
        dual_feasible = True
        
        #for <= constraints the shadow prices should be non negative
        # Check capacity constraint duals (should be non-negative)
        for t in range(1, T+1):
            if capacity_duals[t] < 0:
                dual_feasible = False
                print(f"Capacity dual for period {t} is negative: {capacity_duals[t]}")
                break
        
        # Check production limit duals (should be non-negative)
        if dual_feasible:
            for (i,t) in prod_limit_duals:
                if prod_limit_duals[(i,t)] < 0:
                    dual_feasible = False
                    print(f"Production limit dual for item {i}, period {t} is negative: {prod_limit_duals[(i,t)]}")
                    break
        
        if dual_feasible:
            # Add optimality cut
            master.addConstr(
                theta >= subprob.ObjVal + 
                quicksum(capacity_duals[t] * 
                        quicksum(data.item_requirements[i, 1] * (y[i,t] - y_vals[i,t]) 
                                for i in range(1, O+1)) 
                        for t in range(1, T+1)) +
                quicksum(prod_limit_duals[(i,t)] * 
                        (quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * 
                        (y[i,t] - y_vals[i,t])
                        for i in range(1, O+1) for t in range(1, T+1))
            )
        else:
            # Add feasibility cut
            master.addConstr(
                quicksum(capacity_duals[t] * 
                        quicksum(data.item_requirements[i, 1] * (y[i,t] - y_vals[i,t]) 
                                for i in range(1, O+1)) 
                        for t in range(1, T+1)) +
                quicksum(prod_limit_duals[(i,t)] * 
                        (quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * 
                        (y[i,t] - y_vals[i,t])
                        for i in range(1, O+1) for t in range(1, T+1)) <= 0
            )
        
        # Always reoptimize after adding a cut
        master.optimize()
        
        # Update best solution if we found a better one (lower objective for minimization)
        if master.ObjVal < best_obj:
            best_obj = master.ObjVal
            best_y = y_vals.copy()
            print(f"New best solution found: {best_obj}")
        
        # Check for convergence
        if iteration > 0:
            # You might want to add a tolerance here
            if abs(master.ObjVal - subprob.ObjVal) < 1e-6:
                print("Converged!")
                break
        
        print(f"Master objective: {master.ObjVal}")
        print(f"Subproblem objective: {subprob.ObjVal}")
        print(f"Best objective so far: {best_obj}")
    
    return best_y, best_obj



def create_subprob(y_vals, data):

    T=data.T
    O=data.items

    #now we need to do the subproblem
    sub=gp.Model("Subproblem_dual")

    s=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t


    #this is P1
    sub.setObjective(
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
    GRB.MINIMIZE
    )  


    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr(s[i, t-1]+x[i, t]-s[i, t]==data.demand_forecast[i, t], 
                         name=f"flow_{i}_{t}")

    #constraint 3
    for t in range(1, T+1):
        sub.addConstr(quicksum(x[i, t]*data.item_requirements[i, 1] for i in range(1, O+1)) <= data.capacity[t, 1], 
             name=f"capacity_{t}")
            
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] 
                                          for q in range(t, T+1))) * y_vals[i][t] <= 0, 
                         name=f"prod_limit_{i}_{t}")

    #constraint 7
    for i in range(1, O+1):
        sub.addConstr(s[i, 0] == 0, name=f"init_inv_{i}")

    return sub







    

    

    

In [8]:

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    Benders_decomposition(data, sheet_name, 100)




Iteration 0:
Solving master problem...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 274 columns and 240 nonzeros
Model fingerprint: 0x2ee1fb99
Variable types: 1 continuous, 273 integer (273 binary)
Coefficient statistics:
  Matrix range     [7e+00, 3e+01]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+05, 6e+05]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Master problem status: 2
Master objective: 0.0

Creating and solving subproblem...
G

C:\Users\ricca\AppData\Local\Temp\ipykernel_19156\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_19156\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_df = table_df.fillna(0)
C:\Users\ricca\AppData\Local\Temp\ipykernel_19156\2044414090.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior


CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 512 rows, 546 columns and 1220 nonzeros
Model fingerprint: 0xa158fa2d
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 6e+05]
Presolve removed 272 rows and 306 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
Subproblem status: 3
Subproblem is infeasible. Computing IIS to understand why...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7

## CHALLENGE

Reconsider the 6-periods capacitated single-item uncapacitated lot-sizing problem, and assume now 
that the demand in each period is normally distributed and follows the distribution N(mean = 100, sigma = 20). 
To approximately solve the recourse problem, we first start by discretizing the distribution of the 
random demand 𝒅𝒕 at the end of each period 𝑡. We assume that 𝒅𝒕 takes the realizations (mean ± 𝒌*sigma) 
for 𝑘 = 0,1.5,2.5! Approximate the probability corresponding to each realization.



In [ ]:
#so at each period we know that with a certain prob mean+-k*signma will happen

import scipy.stats as stats


mean = 100
sigma = 20
k = [0, 1.5, 2.5]

# Step 1: Build demand points
demand = [
    mean - k[2]*sigma,
    mean - k[1]*sigma,
    mean - k[0]*sigma,
    mean + k[1]*sigma,
    mean + k[2]*sigma
]  # Resulting in 5 zones

# Step 2: Standardize data points (z-scores)
data_points = [(x - mean) / sigma for x in demand]

# Step 3: Get CDF values (use standard normal)
cdf_values = [stats.norm.cdf(z) for z in data_points]

# Step 4: Compute interval probabilities
interval_probs = [cdf_values[0]]  # left tail
for i in range(1, len(cdf_values)):
    interval_probs.append(cdf_values[i] - cdf_values[i-1])
interval_probs.append(1 - cdf_values[-1])  # right tail

# Print results
print("Demand Points:", demand)
print("Standardized (z):", data_points)
print("CDF Values:", cdf_values)
print("Interval Probabilities:", interval_probs)

i=1
j=1
demand=[0] * 6
while i < 6:
    if i<4:
        #print(i)
        demand[i] = mean - k[i-1] * sigma
        #print(demand[i])
    else:
        demand[i] = mean + k[j] * sigma
        #print(demand[i])
        j=j+1
    i=i+1


demand[1:5] = sorted(demand[1:5])

for i in range (1, 6):
    print(demand[i])


print("\n")

data_points=[0]*6

for i in range (1, 6):
    data_points[i] = ( demand[i] - mean)/sigma

data_points[0:4] = sorted(data_points[1:5])

for i in range (0, 5):
    print(data_points[i])


prob = [0] * 5

# Compute probabilities
probabilities = [stats.norm.cdf( data_points[i], loc=mean, scale=sigma) for i in range (0, 5)]

for i in range(0,5):
    print(probabilities[i])


interval_probs = [probabilities[i] - probabilities[i-1] for i in range(1, len(probabilities))]
interval_probs.insert(0, probabilities[0])  # First interval probability

# Print results
print("Data Points:", data_points)
print("Probabilities:", interval_probs)






Demand Points: [50.0, 70.0, 100, 130.0, 150.0]
Standardized (z): [-2.5, -1.5, 0.0, 1.5, 2.5]
CDF Values: [0.006209665325776132, 0.06680720126885807, 0.5, 0.9331927987311419, 0.9937903346742238]
Interval Probabilities: [0.006209665325776132, 0.06059753594308194, 0.4331927987311419, 0.4331927987311419, 0.060597535943081926, 0.006209665325776159]
